In [1]:
def RandGen16():
    return [GF(2)(x) for x in format(randrange(2^16),'b').zfill(16)]
RandGen16()

[0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0]

In [2]:
import csv
def Read_csv(filename,datatype):
    with open(filename) as csvfile:
        reader = csv.DictReader(csvfile)
        PC_pair = []
        if datatype == 'text':
            for row in reader:
                PC_pair.append((row['plaintext'],row['ciphertext']))
        elif datatype == 'GF2':
            for row in reader:
                PC_pair.append(([GF(2)(i) for i in row['plaintext']],[GF(2)(i) for i in row['ciphertext']]))
        else:
            raise Exception("Please enter correct datatype: 'text' or 'GF2'")
    return PC_pair
PC_pair_group2 = Read_csv('KnownPCpair_Group2.csv','GF2')

In [3]:
PC_pair_group2
PC_pair = PC_pair_group2

In [4]:
from sage.crypto.sbox import SBox
def KeyXOR(data,sK):
    return [x+y for x,y in zip(data,sK)]
def SPN_LinFreAnal(PC_pair, sK5_trial_1_4, sK5_trial_5_8, sK5_trial_9_12, sK5_trial_13_16):
    Sinv = SBox(14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7).inverse()
    [K5_1, K5_2, K5_3, K5_4] = [GF(2)(i) for i in sK5_trial_1_4]
    [K5_5, K5_6, K5_7, K5_8] = [GF(2)(i) for i in sK5_trial_5_8]
    [K5_9, K5_10, K5_11, K5_12] = [GF(2)(i) for i in sK5_trial_9_12]
    [K5_13, K5_14, K5_15, K5_16] = [GF(2)(i) for i in sK5_trial_13_16]
    cnt=0
    for Plaintext, Ciphertext in PC_pair:
        V4_1_4    = KeyXOR(Ciphertext[0:4],[K5_1, K5_2, K5_3, K5_4])
        V4_5_8    = KeyXOR(Ciphertext[4:8],[K5_5, K5_6, K5_7, K5_8])
        V4_9_12   = KeyXOR(Ciphertext[8:12],[K5_9, K5_10, K5_11, K5_12])
        V4_13_16  = KeyXOR(Ciphertext[12:16],[K5_13, K5_14, K5_15, K5_16])
        
        [U4_1, U4_2, U4_3, U4_4] = Sinv(V4_1_4)
        [U4_5, U4_6, U4_7, U4_8] = Sinv(V4_5_8)
        [U4_9, U4_10, U4_11, U4_12] = Sinv(V4_9_12)
        [U4_13, U4_14, U4_15, U4_16] = Sinv(V4_13_16)
        
        P5 = Plaintext[4]
        P8 = Plaintext[7]
        if P5 + P8 + U4_2 + U4_4 + U4_6 + U4_8 + U4_10 + U4_12 + U4_14 + U4_16 == 0:
            cnt += 1
    return 1.0*cnt/len(PC_pair)-1/2
result = {}
for i in range(16):
    for j in range(16):
        sK_trial_5_8 = '{:04b}'.format(i)
        sK_trial_9_12 = '{:04b}'.format(j)
        result['i=%d,j=%d'%(i,j)]=SPN_LinFreAnal(PC_pair, '0101', sK_trial_5_8, sK_trial_9_12 ,'0011')

In [5]:
result

{'i=0,j=0': -0.00879999999999997,
 'i=0,j=1': -0.00240000000000001,
 'i=0,j=2': 0.00960000000000005,
 'i=0,j=3': 0.0109000000000000,
 'i=0,j=4': 0.0331000000000000,
 'i=0,j=5': 0.0173000000000000,
 'i=0,j=6': -0.0256000000000000,
 'i=0,j=7': -0.0395000000000000,
 'i=0,j=8': -0.0100000000000000,
 'i=0,j=9': -0.0239000000000000,
 'i=0,j=10': 0.0322000000000000,
 'i=0,j=11': 0.0164000000000000,
 'i=0,j=12': 0.00380000000000003,
 'i=0,j=13': 0.00509999999999999,
 'i=0,j=14': -0.0123000000000000,
 'i=0,j=15': -0.00590000000000002,
 'i=1,j=0': 0.00810000000000000,
 'i=1,j=1': 0.00429999999999997,
 'i=1,j=2': -0.00190000000000001,
 'i=1,j=3': 0.00519999999999998,
 'i=1,j=4': 0.0208000000000000,
 'i=1,j=5': 0.0118000000000000,
 'i=1,j=6': -0.0163000000000000,
 'i=1,j=7': -0.0424000000000000,
 'i=1,j=8': 0.00129999999999997,
 'i=1,j=9': -0.0248000000000000,
 'i=1,j=10': 0.0293000000000000,
 'i=1,j=11': 0.0203000000000000,
 'i=1,j=12': -0.00629999999999997,
 'i=1,j=13': 0.000800000000000023,
 'i

In [13]:
max_v = max(result.values())
min_v = min(result.values())
for key in result.keys():
    if result[key] == max_v or result[key] == min_v:
        print("%s: %.5f"%(key,result[key]))


i=1,j=7: -0.04240
i=3,j=7: 0.06360
